In [31]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from operator import itemgetter

In [32]:
# Load user interests from JSON (replace 'your_interests.json' with your JSON file)
with open('../data/users.json', 'r', encoding='utf-8') as json_file:
    user_data = json.load(json_file)

# Load item data from JSON (replace 'your_items.json' with your JSON file)
with open('../data/campaigns.json', 'r', encoding='utf-8') as json_file:
    item_data = json.load(json_file)

# Load item data from JSON (replace 'your_items.json' with your JSON file)
with open('../data/interactions.json', 'r', encoding='utf-8') as json_file:
    interaction_data = json.load(json_file)

In [33]:
# Chuyển đổi dữ liệu user_data và item_data thành DataFrame
user_df = pd.DataFrame(user_data)
item_df = pd.DataFrame(item_data)
# Tạo DataFrame từ interactions_data
interactions_df = pd.DataFrame(interaction_data)

In [34]:
# Tạo ma trận tương tác người dùng - dự án
interaction_matrix = defaultdict(set)
for interaction in interaction_data:
    user_id, item_id, rating = interaction['user_id'], interaction['item_id'], interaction['interaction']
    interaction_matrix[user_id][item_id] = rating

# Tạo ma trận sở thích người dùng
user_interests = {user['_id']: user['interests'] for user in user_data}

# Tạo ma trận thể loại của dự án
item_genres = {item['_id']: item['genres'] for item in item_data}

# Hàm tính độ tương tự cosine giữa hai người dùng
def cosine_similarity_users(user1, user2):
    common_items = set(interaction_matrix[user1].keys()) & set(interaction_matrix[user2].keys())
    if not common_items:
        return 0.0
    vector1 = [user_interests[user1][genre] if genre in user_interests[user1] else 0 for genre in item_genres[list(common_items)[0]]]
    vector2 = [user_interests[user2][genre] if genre in user_interests[user2] else 0 for genre in item_genres[list(common_items)[0]]]
    return cosine_similarity([vector1], [vector2])[0][0]

# Hàm khuyến nghị dự án cho người dùng dựa trên lọc cộng tác và lọc theo nội dung
def recommend_projects(user_id, num_recommendations=5):
    user_similarities = [(other_user, cosine_similarity_users(user_id, other_user)) for other_user in user_data if other_user['_id'] != user_id]

    # Sắp xếp người dùng tăng dần theo độ tương tự
    user_similarities.sort(key=itemgetter(1), reverse=True)

    recommended_projects = set()
    for other_user, similarity in user_similarities:
        for item_id, rating in interaction_matrix[other_user].items():
            # Lọc ra các dự án mà người dùng chưa tương tác
            if item_id not in interaction_matrix[user_id]:
                recommended_projects.add(item_id)
                if len(recommended_projects) >= num_recommendations:
                    return [project for project in item_data if project['_id'] in recommended_projects]

# Chuyển đổi ma trận tương tác sang tuple có thể hashable
interaction_matrix = {user_id: {item_id: rating for item_id, rating in user_data.items()} for user_id, user_data in interaction_matrix.items()}

# Test: Khuyến nghị 5 dự án cho người dùng đầu tiên
user_id_to_recommend = user_data[0]['_id']
recommended_projects = recommend_projects(user_id_to_recommend, num_recommendations=5)

print("Dự án được khuyến nghị cho người dùng {}:".format(user_id_to_recommend))
for project in recommended_projects:
    print("- {}".format(project['title']))

TypeError: 'set' object does not support item assignment